<a href="https://colab.research.google.com/github/albim72/ML_ZAAWANSOWANY_11/blob/main/DCGAN_mnist_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from  __future__ import absolute_import
from  __future__ import division
from  __future__ import print_function

In [6]:
from tensorflow.keras.layers import Activation,Dense,Input
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape,Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

In [12]:
import numpy as np
import math
import matplotlib.pyplot as plt
import os
import argparse

In [8]:
def build_generator(inputs,labels,image_size):
  """
  konstruowanie modelu generatora
  wejścia są łączone przed warstwą gęstą
  Stos warstw BN-Relu-Cov2DTranspose do generowania fałszywych obrazów
  funkcja aktywacji - sigmoid

  Argumenty:
  inputs  - warstwa wejściowa genezarora - wektor z
  labels - warstwa wejściowa dla wektora OH - nałożenie warunków wejścia
  image_size - docelowy rozmiar jednego bloku (kwadrat)

  Wyjście -> model generatora
  """

  image_resize = image_size//4
  kernel_size = 5
  layer_filters = [128,64,32,1]

  x = concatenate([inputs,labels],axis=1)
  x = Dense(image_resize*image_resize*layer_filters[0])(x)
  x = Reshape((image_resize,image_size,layer_filters[0]))(x)
  for filters in layer_filters:
    if filters>layer_filters[-2]:
      strides=2
    else:
      strides=1

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(filters=filters,
                        kernel_size=kernel_size,
                        strides = strides,
                        padding='same')(x)
  x = Activation('sigmoid')(x)
  generator = Model([inputs,labels],x,name='generator')
  return generator

In [10]:
def build_discriminator(inputs,labels,image_size):
  """
  konstruowanie modelu dyskryminatora
  wejścia są łączone w warstwie gęstej
  Stos warstw LeakyRelu - Conv2D do odróżniania prawdziwych i fałszywych obrazów

  Argumenty:
  inputs  - warstwa wejściowa dyskryminatora - wektor z
  labels - warstwa wejściowa dla wektora OH - nałożenie warunków wejścia
  image_size - docelowy rozmiar jednego bloku (kwadrat)

  Wyjście -> model dyskryminatora
  """
  kernel_size = 5
  layer_filters = [32,64,128,256]

  x = inputs
  y = Dense(image_size*image_size)(labels)
  y = Reshape((image_size,image_size,1))(y)
  x = concatenate([x,y])

  for filters in layer_filters:
    if filters == layer_filters[-1]:
      strides = 1
    else:
      strides = 2
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,
               padding='same')(x)

  x = Flatten()(x)
  x = Dense(1)(x)
  x = Activation('sigmoid')(x)
  discriminator = Model([inputs,labels],x,name='discriminator')
  return discriminator

Trenowanie dyskryminatora i sieci współzawodniczącej (generatora)

In [13]:
def plot_images(generator,
                noise_input,
                noise_class,
                show=False,
                step=0,
                model_name = "gan"):
  os.makedirs(model_name,exist_ok=True)
  filename = os.path.join(model_name,"%05d.png" % step)
  images = generator.predict([noise_input,noise_class])
  print(model_name," labels -> generated images: ", np.argmax(noise_class,axis=1))
  plt.figure(figsize=(2.2,2.2))
  num_images = images.shape[0]
  image_size = images.shape[1]
  rows = int(math.sqrt(noise_input.shape[0]))
  for i in range(num_images):
    plt.subplot(rows,rows,i+1)
    image = np.reshape(images[i], [image_size,image_size])
    plt.imshow(image,cmap='gray')
    plt.axis('off')
  plt.savefig(filename)
  if show:
    plt.show()
  else:
    plt.close('all')

In [14]:
def train(models,data,params):
  """
  naprzemienne trenowanie dyskryminatora i sieci współzawodniczącej (generatora) przez próbki danych.
  1. Trening dyskryminatora -> analiza próbki z poprawnie zaetykietowanymi obrazami i sztucznymi.
  2. Trenowanie generatora -> sztuczne obrazy
  3. Wyjścia dyskryminatora są warunkowane przez etykiety ze zbioru treningowego dla obrazów prawdziwych i etykietami losowymi dla fałszywych
  argumenty:

  models - generator,dyskryminator, model sieci współzawodniczącej
  data - x_train, y_train
  params: parametry sieci
  """

  generator,discriminator,adversarial = models
  x_train, y_train = data
  batch_size, latent_size, train_steps, num_labels, model_name = params

  save_interval = 500
  #wektor szumu do oceny postępów generatora
  noise_input = np.random.uniform(-1.0,1.0,size=[16,latent_size])
  #etykieta dla warunkowania szumu
  noise_class = np.eye(num_labels)[np.arange(0,16) % num_labels]
  train_size = x_train.shape[0]

  print(model_name,"Etykiety dla generowanych obrazów", np.argmax(noise_class,axis=1))

  for i in range(train_steps):
    rand_indexes = np.random.randint(0,train_size,size=batch_size)
    real_images = x_train[rand_indexes]
    real_labels = y_train[rand_indexes]
    noise = np.random.uniform(-1.0,1.0,size=[batch_size,latent_size])
    #losowe etykiety
    fake_labels = np.eye(num_labels)[np.random.choice(num_labels,batch_size)]

    #generowanie sztucznych obrazów opisywanych sztucznymi etykietami
    fake_images = generator.predict([noise,fake_labels])
    #partia danych uczących = 1 prawdziwy + 1 wygenerowany
    x = np.concatenate((real_images,fake_images))
    labels = np.concatenate((real_labels,fake_labels))

    y = np.ones([2*batch_size,1])
    y[batch_size:,:] = 0.0

    #uczenie sieci dyskryminatora
    loss,acc = discriminator.train_on_batch([x,labels],y)
    log = f"{i}: [dicriminator loss: {loss}, accuracy: {acc}]"

    #uczenie sieci współzawodniczącej
    noise = np.random.uniform(-1.0,1.0,size=[batch_size,latent_size])
    fake_labels = np.eye(num_labels)[np.random.choice(num_labels,batch_size)]

    y = np.ones([batch_size,1])

    loss,acc = adversarial.train_on_batch([noise,fake_labels],y)
    log = f"{i}: [adversarial loss: {loss}, accuracy: {acc}]"
    print(log)

    if(i+1) % save_interval == 0:
      plot_images(generator,
                  noise_input = noise_input,
                  noise_class = noise_class,
                  show=False,
                  step=(i+1),
                  model_name = model_name)
  generator.save(model_name + ".h5")
